In [5]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import colorama
Main_Dataset = pd.read_csv('demo.csv')
Main_Dataset = pd.DataFrame(Main_Dataset)
print(Main_Dataset.shape)
Main_Dataset.head(10).style.set_properties(**{'background-color': '#F6E7E8',
                           'color': 'black',
                           'border-color': 'black'})

(3000, 201)
